## 동의어로 증강된 데이터로 실험 진행
+ baseline 모델
+ attention 모델
+ pretrained_emb 모델



In [1]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 40.9 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313498 sha256=ad5f0f4516d3501f9a9d88b730cd5fac24938851dea51277659b1230c256be91
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [2]:
import os
import re
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input

sys.path.append('/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong') # 모듈경로 추가
from utils.preprocessing import *
from utils.models import *

import wandb
from wandb.integration.keras.callbacks import WandbMetricsLogger
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 00_jw (jiwoong-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
INPUT_MAX_LENGTH = 350  # 입력 시퀀스 최대 길이
NUM_WORDS = 7000 # 토큰화에 사용할 단어 갯수
EMBEDDING_DIM = 64  # 임베딩 차원(100차원 사전학습 임베딩)

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13
current_dir = os.getcwd()
csv_dir = os.path.join(current_dir, 'data', 'raw_csv')
aug_csv_dir = os.path.join(current_dir, 'data', 'aug_csv')

train_csv_path = os.path.join(csv_dir, 'merged_train.csv')
aug_csv_path = os.path.join(aug_csv_dir, 'aug_synonym_train_only.csv') # 4개 카테고리에 대해 증강한 데이터(6357개)

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13


## 증강 데이터 합치기

In [5]:
df = pd.read_csv(train_csv_path)
df_aug = pd.read_csv(aug_csv_path)
ordered_columns = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화'] # kaggle에 명시된 순서로 재배치

conversations = df['conversation']
labels = pd.get_dummies(df['class'])
labels = labels[ordered_columns]

conversations_aug = df_aug['conversation']
labels_aug = pd.get_dummies(df_aug['class'])

# '일반 대화' 클래스는 증강하지 않았기 때문에 컬럼을 만들고 모두 False으로 채움(형태를 맞춰주기 위함)
for col in ordered_columns:
    if col not in labels_aug.columns:
        labels_aug[col] = False
labels_aug = labels_aug[ordered_columns]

# 훈련셋,검증셋 분리
X_train_raw, X_val_raw, y_train, y_val = train_test_split(conversations, labels, test_size=0.2, stratify=labels, random_state=42)

In [6]:
X_train_combined = pd.concat([X_train_raw, conversations_aug], ignore_index=True)
y_train = pd.concat([y_train, labels_aug], ignore_index=True)

#셔플
combined_df = pd.concat([X_train_combined, y_train], axis=1)
shuffled_df = combined_df.sample(frac=1).reset_index(drop=True)

# 다시 분리
X_train_combined = shuffled_df.iloc[:, 0]
y_train = shuffled_df.iloc[:, 1:]

## 데이터 전처리

In [7]:
# 토크나이저 생성
tokenizer = create_tokenizer(X_train_combined, NUM_WORDS)
print("생성된 어휘 수:", len(tokenizer.word_index))

생성된 어휘 수: 52498


In [8]:
import pickle

# 저장
with open("/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong/data/tokenizer_aug_synonym.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# # 불러오기
# with open("/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong/data/tokenizer_aug_synonym.pkl", "rb") as f:
#     tokenizer = pickle.load(f)

In [9]:
# 전처리 수행
X_train = preprocessing(X_train_combined, tokenizer, INPUT_MAX_LENGTH)
X_val = preprocessing(X_val_raw, tokenizer, INPUT_MAX_LENGTH)

In [10]:
import pickle

# 전처리된 데이터 저장
with open('/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong/data/preprocessed_data_aug_synonym.pkl', 'wb') as f:
    pickle.dump({'X_train': X_train, 'X_val': X_val, 'y_train': y_train, 'y_val': y_val}, f)

# # 불러오기
# with open('preprocessed_data.pkl', 'rb') as f:
#     data = pickle.load(f)
#     X_train = data['X_train']
#     X_val = data['X_val']
#     y_train = data['y_train']
#     y_val = data['y_val']

## 모델학습

### baseline 모델

In [11]:
model = build_model_baseline(NUM_WORDS, INPUT_MAX_LENGTH, EMBEDDING_DIM)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 350, 64)        │       448,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 514,757 (1.96 MB)

 Trainable params: 514,757 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# F1Score 평가 지표
f1 = F1Score(average='micro', name='f1')  # micro f1: 전체 클래스 f1score 평균

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

es_callback = EarlyStopping(
    monitor='val_f1',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

mc_callback = ModelCheckpoint(
  f'./notebooks/jiwoong/models/baseline_aug_synonym.keras',
  monitor="val_f1",
  save_best_only=True,
  mode="max"
)

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"exp_baseline_aug_synonym", # 실험 이름
    notes="baseline모델에 synonym으로 증강한 데이터를 사용하여 학습", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "exp_baseline_aug_synonym",
        "general_conversation_type": "LLM에서 생성한 대화 데이터", # 일반 데이터 타입
        "architecture": "BiLSTM(64)",
    },
)

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[es_callback, mc_callback, WandbMetricsLogger()] # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

# 최적의 모델로 검증세트 예측
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

Epoch 1/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - accuracy: 0.4091 - f1: 0.4091 - loss: 1.2962 - val_accuracy: 0.7877 - val_f1: 0.7877 - val_loss: 0.6014
Epoch 2/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.8715 - f1: 0.8715 - loss: 0.4226 - val_accuracy: 0.8890 - val_f1: 0.8890 - val_loss: 0.3523
Epoch 3/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.9684 - f1: 0.9684 - loss: 0.1374 - val_accuracy: 0.8869 - val_f1: 0.8869 - val_loss: 0.3532
Epoch 4/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.9851 - f1: 0.9851 - loss: 0.0694 - val_accuracy: 0.9181 - val_f1: 0.9181 - val_loss: 0.3226
Epoch 5/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.9884 - f1: 0.9884 - loss: 0.0543 - val_accuracy: 0.8782 - val_f1: 0.8782 - val_loss: 0.5351
Epoch 6/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9775 - f1: 0.9775 - loss: 0.0901 - val_accuracy: 0.9030 - val_f1: 0.9030 - val_loss: 0.3726
Epoch 7/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 9s 

accuracy,▁
epoch/accuracy,▁▆█████
epoch/epoch,▁▂▃▅▆▇█
epoch/f1,▁▆█████
epoch/learning_rate,▁▁▁▁▁▁▁
epoch/loss,█▃▂▁▁▁▁
epoch/val_accuracy,▁▆▆█▆▇█
epoch/val_f1,▁▆▆█▆▇█
epoch/val_loss,█▂▂▁▆▂▃
extortion_f1,▁
general_conversation_f1,▁


In [13]:
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true_labels, y_pred_labels))
print(classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns))

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
[[153   9   5  11   1]
 [  5 178   3   9   0]
 [  1   0 187   5   1]
 [  7   5  12 178   0]
 [  0   1   1   0 156]]
              precision    recall  f1-score   support

       협박 대화       0.92      0.85      0.89       179
       갈취 대화       0.92      0.91      0.92       195
 직장 내 괴롭힘 대화       0.90      0.96      0.93       194
   기타 괴롭힘 대화       0.88      0.88      0.88       202
       일반 대화       0.99      0.99      0.99       158

    accuracy                           0.92       928
   macro avg       0.92      0.92      0.92       928
weighted avg       0.92      0.92      0.92       928



### attention 모델

In [14]:
model_attention = build_model_attention(NUM_WORDS, INPUT_MAX_LENGTH, EMBEDDING_DIM)
model_attention.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 350, 64)        │       448,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 350, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 128)            │           478 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 515,235 (1.97 MB)

 Trainable params: 515,235 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# F1Score 평가 지표
f1 = F1Score(average='micro', name='f1')  # micro f1: 전체 클래스 f1score 평균

model_attention.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

es_callback = EarlyStopping(
    monitor='val_f1',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

mc_callback = ModelCheckpoint(
  f'./notebooks/jiwoong/models/model_attention_aug_synonym.keras',
  monitor="val_f1",
  save_best_only=True,
  mode="max"
)

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"exp_attention_aug_synonym", # 실험 이름
    notes="attention모델에 synonym으로 증강한 데이터를 사용하여 학습", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "exp_attention_aug_synonym",
        "general_conversation_type": "LLM에서 생성한 대화 데이터", # 일반 데이터 타입
        "architecture": "BiLSTM(64), Attention",
    },
)

history = model_attention.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[es_callback, mc_callback, WandbMetricsLogger()] # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

# 최적의 모델로 검증세트 예측
y_pred = model_attention.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

Epoch 1/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 14s 34ms/step - accuracy: 0.3067 - f1: 0.3067 - loss: 1.4582 - val_accuracy: 0.7112 - val_f1: 0.7112 - val_loss: 0.7133
Epoch 2/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.8075 - f1: 0.8075 - loss: 0.5461 - val_accuracy: 0.8696 - val_f1: 0.8696 - val_loss: 0.3871
Epoch 3/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.9329 - f1: 0.9329 - loss: 0.2359 - val_accuracy: 0.8998 - val_f1: 0.8998 - val_loss: 0.3151
Epoch 4/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.9686 - f1: 0.9686 - loss: 0.1149 - val_accuracy: 0.9181 - val_f1: 0.9181 - val_loss: 0.2955
Epoch 5/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.9797 - f1: 0.9797 - loss: 0.0804 - val_accuracy: 0.9116 - val_f1: 0.9116 - val_loss: 0.3510
Epoch 6/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.9874 - f1: 0.9874 - loss: 0.0540 - val_accuracy: 0.9213 - val_f1: 0.9213 - val_loss: 0.3266
Epoch 7/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 1

accuracy,▁
epoch/accuracy,▁▆▇██████
epoch/epoch,▁▂▃▄▅▅▆▇█
epoch/f1,▁▆▇██████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▁▁▁▁▁▁
epoch/val_accuracy,▁▆▇███▇▇█
epoch/val_f1,▁▆▇███▇▇█
epoch/val_loss,█▃▁▁▂▂▂▃▂
extortion_f1,▁
general_conversation_f1,▁


In [16]:
y_pred = model_attention.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true_labels, y_pred_labels))
print(classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns))

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[157   8   2  11   1]
 [  5 180   2   8   0]
 [  3   1 181   9   0]
 [ 11   3   1 187   0]
 [  0   7   0   1 150]]
              precision    recall  f1-score   support

       협박 대화       0.89      0.88      0.88       179
       갈취 대화       0.90      0.92      0.91       195
 직장 내 괴롭힘 대화       0.97      0.93      0.95       194
   기타 괴롭힘 대화       0.87      0.93      0.89       202
       일반 대화       0.99      0.95      0.97       158

    accuracy                           0.92       928
   macro avg       0.93      0.92      0.92       928
weighted avg       0.92      0.92      0.92       928



### pretrained_emb 모델

In [17]:
model_pretrained_emb = build_model_pretrained_embedding(tokenizer, NUM_WORDS, INPUT_MAX_LENGTH, 100) # 임베딩 차원: 100
model_pretrained_emb.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 350, 100)       │       700,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785,225 (3.00 MB)

 Trainable params: 785,225 (3.00 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# F1Score 평가 지표
f1 = F1Score(average='micro', name='f1')  # micro f1: 전체 클래스 f1score 평균

model_pretrained_emb.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

es_callback = EarlyStopping(
    monitor='val_f1',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

mc_callback = ModelCheckpoint(
  f'./notebooks/jiwoong/models/model_pretrained_emb_aug_synonym.keras',
  monitor="val_f1",
  save_best_only=True,
  mode="max"
)

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"exp_pretrained_emb_aug_synonym", # 실험 이름
    notes="pretrained_emb를 사용한 모델에 synonym으로 증강한 데이터를 사용하여 학습", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "exp_pretrained_emb_aug_synonym",
        "general_conversation_type": "LLM에서 생성한 대화 데이터", # 일반 데이터 타입
        "architecture": "BiLSTM(64), Pretrained_emb",
    },
)

history = model_pretrained_emb.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[es_callback, mc_callback, WandbMetricsLogger()] # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

# 최적의 모델로 검증세트 예측
y_pred = model_pretrained_emb.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

Epoch 1/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.4581 - f1: 0.4581 - loss: 1.2479 - val_accuracy: 0.8502 - val_f1: 0.8502 - val_loss: 0.4999
Epoch 2/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.9033 - f1: 0.9033 - loss: 0.3290 - val_accuracy: 0.9052 - val_f1: 0.9052 - val_loss: 0.3070
Epoch 3/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.9638 - f1: 0.9638 - loss: 0.1357 - val_accuracy: 0.8976 - val_f1: 0.8976 - val_loss: 0.3844
Epoch 4/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.9859 - f1: 0.9859 - loss: 0.0660 - val_accuracy: 0.9106 - val_f1: 0.9106 - val_loss: 0.3401
Epoch 5/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.9896 - f1: 0.9896 - loss: 0.0444 - val_accuracy: 0.9224 - val_f1: 0.9224 - val_loss: 0.3322
Epoch 6/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.9938 - f1: 0.9938 - loss: 0.0313 - val_accuracy: 0.9073 - val_f1: 0.9073 - val_loss: 0.3984
Epoch 7/40
314/314 ━━━━━━━━━━━━━━━━━━━━ 10s 

accuracy,▁
epoch/accuracy,▁▇▇█████
epoch/epoch,▁▂▃▄▅▆▇█
epoch/f1,▁▇▇█████
epoch/learning_rate,▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▁▁▁▁▁
epoch/val_accuracy,▁▆▆▇█▇▇▇
epoch/val_f1,▁▆▆▇█▇▇▇
epoch/val_loss,█▁▄▂▂▄▄▅
extortion_f1,▁
general_conversation_f1,▁


In [19]:
y_pred = model_pretrained_emb.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true_labels, y_pred_labels))
print(classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns))

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[[149  11   3  15   1]
 [  4 180   5   6   0]
 [  1   1 188   4   0]
 [  6   5   8 183   0]
 [  0   2   0   0 156]]
              precision    recall  f1-score   support

       협박 대화       0.93      0.83      0.88       179
       갈취 대화       0.90      0.92      0.91       195
 직장 내 괴롭힘 대화       0.92      0.97      0.94       194
   기타 괴롭힘 대화       0.88      0.91      0.89       202
       일반 대화       0.99      0.99      0.99       158

    accuracy                           0.92       928
   macro avg       0.93      0.92      0.92       928
weighted avg       0.92      0.92      0.92       928

